In [ ]:
#importing libraries
from __future__ import print_function
#Keras is a great high-level library which allows anyone to create powerful machine learning models in minutes.
import keras
#Keras has this ImageDataGenerator class which allows the users to perform image augmentation on the fly in a very easy way.
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
import  PIL as pillow
from PIL import Image
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32

#input dataset

train_data_dir = r'C:\Users\HP\Desktop\EmotionDetection\Dataset\train'
validation_data_dir = r'C:\Users\HP\Desktop\EmotionDetection\Dataset\validation'

#Image Preprocessing->Generate batches of tensor image data with real-time data augmentation. The data will be looped over
#(in batches).
#Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for
#training models, without actually collecting new data. Data augmentation techniques such as cropping, padding, and horizontal 
#flipping are commonly used to train large neural networks

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    width_shift_range=0.4,
                    height_shift_range=0.4,
                    horizontal_flip=True,
                    fill_mode='nearest')

#Rescale is a value by which we will multiply the data before any other processing. Our original images consist in RGB 
#coefficients in the 0-255, but such values would be too high for our model to process (given a typical learning rate),
#so we target values between 0 and 1 instead by scaling with a 1/255.


validation_datagen = ImageDataGenerator(rescale=1./255)

#The ImageDataGenerator class has method flow_from_directory()  to read the images from a big numpy array and folders containing images.

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        color_mode='grayscale',
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            color_mode='grayscale',
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical',
                            shuffle=True)


model = Sequential()

#2D convolution layer (e.g. spatial convolution over images).
#This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of output
#
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))     #Activation function _>Exponential linear unit.In artificial neural networks, 
model.add(BatchNormalization())  #the activation function of a node defines the output of that node given an input or set of inputs.
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())#Normalize the activations of the previous layer at each batch, i.e. applies a transformation
#that maintains the mean activation close to 0 and the activation standard deviation close to 1.
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))#Initializations define the way to set the initial random weights of Keras layers.
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))#Pooling mainly helps in extracting sharp and smooth features. It is also done to reduce variance and computations. Max-pooling helps in extracting low-level features like edges, points, etc
model.add(Dropout(0.2))# Dropout is an approach to regularization in neural networks which helps reducing interdependent learning amongst the neurons.

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())         #sumary of the model



from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#A callback is a set of functions to be applied at given stages of the training procedure. 
#You can use callbacks to get a view on internal states and statistics of the model during training.




#An H5 file is a data file saved in the Hierarchical Data Format (HDF). It contains multidimensional arrays of scientific
#data. H5 files are commonly used in aerospace, physics, engineering, finance, academic research, genomics, astronomy, electronics instruments, and medical fields.
checkpoint = ModelCheckpoint('EmotionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',  #Early stopping is basically stopping the training once your loss starts to increase (or in other words validation accuracy starts to decrease).
                          min_delta=0,
                          patience=9,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',    #Reduce learning rate when a metric has stopped improving. 
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(                      #.fit_generator  function assumes there is an underlying function that is generating the data for it.
                train_generator,
                steps_per_epoch=nb_train_samples/batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples/batch_size)





Using TensorFlow backend.


Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
___________________________________